In [1]:
import pandas as pd
import numpy as np
from datetime import time
from backtesting import Backtest, Strategy

def add_vwap(
    df: pd.DataFrame,
    time_col: str | None = None,
    price_col: str = "Close",
    vol_col: str = "Volume",
    **parse_kwargs                
) -> pd.DataFrame:
    df = df.copy()

    # 1 ─ ensure DatetimeIndex
    if time_col is not None:
        df[time_col] = pd.to_datetime(df[time_col], errors="coerce", **parse_kwargs)
        df = df.set_index(time_col)

    if not isinstance(df.index, pd.DatetimeIndex) or df.index.hasnans:
        raise TypeError("Index (or `time_col`) must be datetime-like and parse without NaT")

    # 2 ─ price to weight by volume
    tp = ((df["High"] + df["Low"] + df["Close"]) / 3) if price_col.lower() == "typical" else df[price_col]

    # 3 ─ VWAP per calendar day
    day = df.index.normalize()
    cum_vol = df[vol_col].groupby(day).cumsum()
    cum_pv  = (tp * df[vol_col]).groupby(day).cumsum()
    df["VWAP"] = cum_pv / cum_vol
    return df

C:\Users\josep\anaconda3\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
def add_weekly_vwap(
    df: pd.DataFrame,
    *,
    time_col: str | None = None,
    price_col: str = "Close",
    vol_col:   str = "Volume",
    freq:      str = "D",           
    **parse_kwargs,                 
) -> pd.DataFrame:
    """
    Append a VWAP column that resets every *freq* period.

    Parameters
    ----------
    df : pd.DataFrame
        Must contain `price_col`, `vol_col`, and (optionally) High/Low if
        price_col="Typical".
    time_col : str | None
        Pass the name of the timestamp column if the index is not already
        Datetime-like.
    price_col : {"Close", "Typical", ...}
        Price to weight by volume.  "Typical" uses (High+Low+Close)/3.
    vol_col : str
        Volume column name.
    freq : str, default "D"
        Pandas offset alias that defines VWAP reset boundary.
        Examples: "D" (daily), "W-MON" (weekly ending Monday), "M" (monthly).
    **parse_kwargs
        Extra arguments passed to `pd.to_datetime` when parsing `time_col`.

    Returns
    -------
    pd.DataFrame
        Copy of `df` with an added 'VWAP' column.
    """
    df = df.copy()

    # -- 1. ensure DatetimeIndex 
    if time_col is not None:
        df[time_col] = pd.to_datetime(df[time_col], errors="coerce", **parse_kwargs)
        df = df.set_index(time_col)

    if not isinstance(df.index, pd.DatetimeIndex) or df.index.hasnans:
        raise TypeError("Index (or `time_col`) must be datetime-like and parse without NaT")

    # -- 2. choose the price series 
    if price_col.lower() == "typical":
        tp = (df["High"] + df["Low"] + df["Close"]) / 3
    else:
        tp = df[price_col]

    # -- 3. labels defining each VWAP bucket 
    # floor to the start of each period (e.g. Monday 00:00 if freq="W-MON")
    labels = df.index.to_period(freq).to_timestamp()

    # cumulative sums within each period
    cum_vol = df[vol_col].groupby(labels).cumsum()
    cum_pv  = (tp * df[vol_col]).groupby(labels).cumsum()

    df["VWAP"] = cum_pv / cum_vol
    return df


In [3]:
class VWAPBreakout(Strategy):
    """
    • Long  when Close crosses above VWAP
    • Short when Close crosses below VWAP
    Optional filter: take longs only if Close > day’s Open (bull bias)
    Always flat by 15:45‒16:00 ET bar
    """

    # --- configurable parameters 
    intraday_close_time = time(15, 45)  # last bar before close (interval = 15m)
    atr_stop = 1.5  # e.g. 1.5 to use ATR trailing stop, None = no stop

    def init(self):
        # pre-compute ATR if a trailing stop is requested
        if self.atr_stop:
            self.atr = self.I(self._atr, self.data.High, self.data.Low, self.data.Close, 14)

    # ---- utility: ATR (simple True Range MA)
    @staticmethod
    def _atr(h, l, c, n):
        tr = np.maximum.reduce([h[1:] - l[1:], abs(h[1:] - c[:-1]), abs(l[1:] - c[:-1])])
        atr = pd.Series(tr).rolling(n).mean()
        return np.append([np.nan], atr)  # align length

    # -------------------------------------------------------
    def next(self):
        #i = len(self.data.Close) - 1  # current bar index

        close = self.data.Close[-1]
        vwap  = self.data.VWAP[-1]

        # --- determine daily open price for filters ----------
        current_day = self.data.index[-1].date()
        day_open = self.data.Open[self.data.index.date == current_day][0]

        # -------- entry logic --------------------------------
        if not self.position:

            # LONG
            if (close > vwap) and (close > day_open): #and close>self.data.Open[-1]
                self.buy()

            # SHORT
            elif (close < vwap) and (close < day_open): # and close<self.data.Open[-1]
                self.sell()

        if self.position and self.atr_stop:          # <-- note lower-case .position
            price = self.data.Close[-1]     
            atr = self.atr[-1]
            trail = self.atr_stop * atr

            # tighten stop on every active trade
            for trade in self.trades:
                if trade.is_long:
                    new_sl = price - trail
                    if trade.sl is None or new_sl > trade.sl:
                        trade.sl = new_sl
                else:                      # short trade
                    new_sl = price + trail
                    if trade.sl is None or new_sl < trade.sl:
                        trade.sl = new_sl

        # -------- intraday exit: flatten before close 
        if self.position:
            bar_time = self.data.index[-1].time()
            if bar_time >= self.intraday_close_time:
                self.position.close()

In [9]:
df = pd.read_csv("TSLA.USUSD_2023_a_hoyFINAL.csv")
# >>> ¡ESTA LÍNEA ES CRUCIAL! <<<
df['Gmt time'] = pd.to_datetime(df['Gmt time'], format='%Y-%m-%d %H:%M:%S')

df = add_vwap(
            df,
            time_col="Gmt time",
            dayfirst=True,                         # parses 03.06.2024 correctly
            format="%d.%m.%Y %H:%M:%S.%f"          # speeds up parsing, too
            )
# df = add_weekly_vwap(
#         df[100:],
#         time_col="Gmt time",               # ← let the helper parse & index it
#         dayfirst=True,                     # 03.06.2024 = 3 June 2024
#         format="%d.%m.%Y %H:%M:%S.%f",     # exact pattern = faster/no NaT
#         freq="W-FRI"                       # weekly VWAP, week ends Friday
# )

bt = Backtest(
    df[1000:5000],
    VWAPBreakout,
    cash=100_000,
    commission=0.000,  # e.g. 0.001 for 0.1% commission
    exclusive_orders=True,  # only one position at a time
    trade_on_close=True,    # act on bar close prices
)

import matplotlib.pyplot as plt

stats = bt.run()
print(stats)       # show key stats
bt.plot(show_legend=False)

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

C:\Users\josep\AppData\Local\Temp\ipykernel_11140\1202335922.py:30: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = bt.run()


Start                     2023-02-28 17:30:00
End                       2023-10-09 18:15:00
Duration                    223 days 00:45:00
Exposure Time [%]                        81.7
Equity Final [$]                 127378.61125
Equity Peak [$]                  149173.47761
Return [%]                           27.37861
Buy & Hold Return [%]                28.21845
Return (Ann.) [%]                    48.20627
Volatility (Ann.) [%]                46.79593
CAGR [%]                             31.44593
Sharpe Ratio                          1.03014
Sortino Ratio                         2.48127
Calmar Ratio                          1.95024
Alpha [%]                             29.2903
Beta                                 -0.06775
Max. Drawdown [%]                   -24.71816
Avg. Drawdown [%]                    -1.96486
Max. Drawdown Duration       76 days 03:30:00
Avg. Drawdown Duration        3 days 19:02:00
# Trades                                 1204
Win Rate [%]                      

GridPlot(id='p2184', ...)

In [6]:
# --------------------------------------------------------
# Sweep atr_stop from 0.5 to 3.0 in 0.25 steps
# --------------------------------------------------------
stats_best, heatmap = bt.optimize(
    atr_stop = [round(x, 2) for x in np.arange(1, 2.51, 0.25)],
    maximize = "Sharpe Ratio",
    return_heatmap = True,      # keep all runs, not just the best
)

# ------------ best run ------------
best_atr  = stats_best._strategy.atr_stop
best_ret  = stats_best["Return [%]"]
best_sharpe = stats_best["Sharpe Ratio"]

print("=== Best parameter set ===")
print(f"atr_stop      : {best_atr:.2f}")
print(f"Return [%]    : {best_ret:.2f}")
print(f"Sharpe Ratio  : {best_sharpe:.2f}")

C:\Users\josep\anaconda3\Lib\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/7 [00:00<?, ?it/s]

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

C:\Users\josep\anaconda3\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)
C:\Users\josep\anaconda3\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)
C:\Users\josep\anaconda3\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)
C:\Users\josep\anaconda3\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.ru

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

=== Best parameter set ===
atr_stop      : 1.00
Return [%]    : 31.17
Sharpe Ratio  : 1.22


C:\Users\josep\anaconda3\Lib\site-packages\backtesting\backtesting.py:1545: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = self.run(**dict(zip(heatmap.index.names, best_params)))


In [7]:
# ------------ all runs ------------
# heatmap is a Series (single parameter) whose index is atr_stop
summary = (
    heatmap
    .rename("Sharpe Ratio")      # put metric in a column
    .reset_index()               # bring atr_stop out of the index
    .rename(columns={"index": "atr_stop"})
    .sort_values("atr_stop")
)

# For cumulative return we need to run once per value;
# easiest: loop over summary index and pull the stat from bt.run()
returns = []
for val in summary["atr_stop"]:
    res = bt.run(atr_stop=val)
    returns.append(res["Return [%]"])
summary["Return [%]"] = returns

print("\n=== Full sweep results ===")
print(summary.to_string(index=False))

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

C:\Users\josep\AppData\Local\Temp\ipykernel_11140\3454991667.py:15: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  res = bt.run(atr_stop=val)


Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

C:\Users\josep\AppData\Local\Temp\ipykernel_11140\3454991667.py:15: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  res = bt.run(atr_stop=val)


Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

C:\Users\josep\AppData\Local\Temp\ipykernel_11140\3454991667.py:15: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  res = bt.run(atr_stop=val)


Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]


=== Full sweep results ===
 atr_stop  Sharpe Ratio  Return [%]
     1.00      1.220789   31.170990
     1.25      1.060027   27.592498
     1.50      1.030138   27.378611
     1.75      1.083335   31.199036
     2.00      0.899692   24.569857
     2.25      0.731160   18.895861
     2.50      0.731753   18.345433


C:\Users\josep\AppData\Local\Temp\ipykernel_11140\3454991667.py:15: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  res = bt.run(atr_stop=val)


In [8]:
bt.plot()

GridPlot(id='p1780', ...)